In [2]:
#import matplotlib.pyplot as plt # for plotting maps
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np
import time
import os
from zipfile import ZipFile
import zipfile
from datetime import date
from datetime import datetime
from pytz import timezone
import shutil
import multiprocessing as mp

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None) 

## Internal validation function

check for
- totals the same, excluding place?
    For non-place levels:
    - "ALL P1,P2,P3,P4,P5,H1 TOTALS ARE THE SAME"
    - "do not match the state totals."
- are place totals greater or less than?
    - "is LESS than the state total."
    - "is MORE than the state total."
- row count - 
    - "There are the same number of rows in the shapefile and the PL data."
    - "There are NOT the same number of rows in the shapefile and the PL data."
    - "THE QUERIED DATA, SHAPEFILE, AND CREATED GEODATAFRAME ARE ALL THE SAME LENGTH"
    - "THE QUERIED DATA, SHAPEFILE, AND CREATE GEODATAFRAME ARE NOT ALL THE SAME LENGTH."
- no data exists cases
    - "There is no shapefile available at the" --- if this occurs print full line
    - "THERE IS NO DATA AT THIS LEVEL FOR"

In [3]:
# Download from s3 and run locally
def verify_documentation(filepath, textfile, equalpath, notequalpath):
    with open(filepath+textfile, 'r') as f:
        contents = f.read()
        lines = f.readlines()
        
        geog_totpop_equal = pd.Series(contents).str.contains("ALL P1,P2,P3,P4,P5,H1 TOTALS ARE THE SAME")
        geog_totpop_notequal = pd.Series(contents).str.contains("do not match the state totals.")
        row_count_equal = pd.Series(contents).str.contains("THE QUERIED DATA, SHAPEFILE, AND CREATED GEODATAFRAME ARE ALL THE SAME LENGT")
        row_count_notequal = pd.Series(contents).str.contains(" NOT ALL")
        no_data_for_level = pd.Series(contents).str.contains("THERE IS NO DATA AT THIS LEVEL FOR")
        place_sum_equal = pd.Series(contents).str.contains("should be the SAME as the state total")
        place_sum_greater = pd.Series(contents).str.contains("is MORE than the state total.")
        place_sum_less = pd.Series(contents).str.contains("is LESS than the state total.")
        
        geog_totpop_notequal_str = "do not match the state totals."
        row_count_notequal_str = "NOT ALL "
        no_data_for_level_str = "THERE IS NO DATA AT THIS LEVEL FOR"
        place_sum_greater_str = ("is MORE than the state total.")
        place_sum_less_str = ("is LESS than the state total.")

        sa = textfile[0:2]
        #EQUAL
        if (geog_totpop_equal.bool()==True)&(geog_totpop_notequal.bool()==False)&(row_count_equal.bool()==True)&(row_count_notequal.bool()==False)&(place_sum_greater.bool()==False):
            f = open(filepath+textfile)
            lines = f.readlines()
            for line in lines:
                if (no_data_for_level_str in line):
                    print(line) 
            shutil.copy(filepath+textfile, equalpath+textfile)
            print(sa, 'has the same total in both the state sum and all geog level sums checked and the same row count')
                
        #NOT EQUAL
        else:
            print(sa, 'something did not match up')
            if place_sum_greater.bool() == True:
                print('Place sum is MORE than state total in at least one case.')
            f = open(filepath+textfile)
            lines = f.readlines()
            for line in lines:
                if (geog_totpop_notequal_str in line)|(row_count_notequal_str in line)|(no_data_for_level_str in line):
                    print(line)
            shutil.copy(filepath+textfile, notequalpath+textfile)
        
    f.close()
 

In [5]:
#Define variables to run validation locally
filepath = './PL_2020/documentation/'
equalpath = './PL_2020/check/equal/'
notequalpath = './PL_2020/check/notequal/'

#Run function for every state's txt file
for textfile in os.listdir(filepath):
    if textfile[0]!='.':
        verify_documentation(filepath, textfile, equalpath, notequalpath)

AL has the same total in both the state sum and all geog level sums checked and the same row count
ND has the same total in both the state sum and all geog level sums checked and the same row count
OH has the same total in both the state sum and all geog level sums checked and the same row count
IN has the same total in both the state sum and all geog level sums checked and the same row count
AR has the same total in both the state sum and all geog level sums checked and the same row count
CA has the same total in both the state sum and all geog level sums checked and the same row count
ME something did not match up
2021-08-15 01:11:41.054493-04:00	 THE QUERIED DATA, SHAPEFILE, AND CREATE GEODATAFRAME ARE NOT ALL THE SAME LENGHT.

KY has the same total in both the state sum and all geog level sums checked and the same row count
2021-08-14 23:53:05.825733-04:00	 THERE IS NO DATA AT THIS LEVEL FOR ne

NE has the same total in both the state sum and all geog level sums checked and the sam

In [4]:
#Define variables to run validation locally
filepath = './PL_2020/check/block_internal_spot_check/'
equalpath = './PL_2020/check/block_equal/'
notequalpath = './PL_2020/check/block_notequal/'
#os.mkdir(equalpath)
#os.mkdir(notequalpath)

#Run function for every state's txt file
for textfile in os.listdir(filepath):
    if textfile[0]!='.':
        verify_documentation(filepath, textfile, equalpath, notequalpath)

AL something did not match up
2021-08-30 19:26:25.563377-04:00	 The totals of b do not match the state totals.

2021-08-30 19:30:51.292365-04:00	 The totals of b do not match the state totals.

2021-08-30 19:35:38.120794-04:00	 The totals of b do not match the state totals.

2021-08-30 19:40:15.119425-04:00	 The totals of b do not match the state totals.

2021-08-30 19:45:06.538484-04:00	 The totals of b do not match the state totals.

ND something did not match up
2021-08-30 19:57:36.824128-04:00	 The totals of b do not match the state totals.

2021-08-30 20:00:59.677256-04:00	 The totals of b do not match the state totals.

2021-08-30 20:03:38.972452-04:00	 The totals of b do not match the state totals.

2021-08-30 20:05:55.519103-04:00	 The totals of b do not match the state totals.

2021-08-30 20:08:42.382816-04:00	 The totals of b do not match the state totals.

OH something did not match up
2021-08-30 17:14:51.012592-04:00	 The totals of b do not match the state totals.

2021-08-

In [ ]:
import multiprocessing as mp 

def my_func(n): 
    print(n)

if __name__ == '__main__':

    state_codes = {
        'WA': '53', 'DE': '10', 'WI': '55', 'WV': '54', 'HI': '15',
        'FL': '12', 'WY': '56', 'NJ': '34', 'NM': '35', 'TX': '48',
        'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
        'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
        'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
        'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
        'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
        'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
        'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
    }
    state_codes = list(state_codes.keys())


    ordered_states = ['TX','AL','AZ','FL','GA','LA','MI','MO','MS','NC','PA','OH','VA','WI', # priority, starting with TX
                'NJ','AK','WA','MT','NV','NM','TN','SC', # tier 2 priority, starting with NJ
                'CA','DE','WV','HI','WY','ND','NE','NY','NH','CO','AR','VT','IL','IN','IA','MA','ID','CT','ME','MD','OK','UT','MN','RI','KS','KY','OR','SD'] #rest, starting with CA

    my_order = ["C:/Spencer/PL2020/raw-pl/" + key + "2020.pl" for i, key in enumerate(ordered_states)]

    p = mp.Pool(10)
    p.map(my_func, my_order)
    p.close()
    p.join()

## Check for leading zeros in RDH file GEOCODE columns

In [84]:
az_dir = ('./PL_2020/check/validation_internal_spot_check/')

for folder in os.listdir(az_dir):
    if not folder[0]=='.':
        for shp in os.listdir(az_dir+folder):
            if shp.endswith('.shp'):
                az_file = gp.read_file(az_dir+folder+'/'+shp)
                print(folder, az_file['GEOCODE'].str.slice(stop = 1).value_counts())

0    4773
Name: GEOCODE, dtype: int64
0    1
Name: GEOCODE, dtype: int64
0    30
Name: GEOCODE, dtype: int64
0    467
Name: GEOCODE, dtype: int64
0    9
Name: GEOCODE, dtype: int64
4    147638
0      7806
Name: GEOCODE, dtype: int64
0    30
Name: GEOCODE, dtype: int64
0    1538
Name: GEOCODE, dtype: int64
0    1765
Name: GEOCODE, dtype: int64
0    15
Name: GEOCODE, dtype: int64


In [87]:
b_az = gp.read_file('./PL_2020/check/validation_internal_spot_check/az_pl2020_b/az_pl2020_b.shp')
#b_az['GEOCODE'].str.slice(stop = 1).value_counts()

In [89]:
b_az.shape

(155444, 386)